# Pivo Recommender

Looking for a new way to compare and recommend beer.  

This notebook scrapes beer reviews from [BeerAdvocate](https://www.beeradvocate.com/) then performs natural language processing on these reviews.  Once a profile of a beer has been created, a similar, semi-similar, or completely different beer can be recommended.

'/Users/johnzupan/mystuff/pivo_rec'

In [ ]:
%cd beerAdvocateScraper
%scrapy crawl reviewScraper

In [2]:
import itertools as it
import os
import pandas as pd
import spacy

In [3]:
nlp = spacy.load('en')

## Phrase Modeling

Phrase modeling is an approach to learning combinations of tokens that together represent meaning multi-word concepts.  These phrase models are developed by looping over the words in the corpus and finding words that appear together more than they should by random chance.  The formula used to determine if whether two tokens $A$ and $B$ constitute a phrase is:

$$
\frac {count(A B) - count_{min}}
{count(A) * count(B)}
*N > threshold
$$

where:
* $count(A)$ is the number of times $A$ appears in the corpus
* $count(B)$ is the number of times $B$ appears in the corpus
* $count(AB)$ is the number of times $AB$ appear in the corpus in that order
* $N$ is the total size of the corpus vocabulary
* $count_{min}$ is a user-defined parameter to ensure that the phrase appears a minimum number of times
* $threshold$ is a user-defined paramter to control how strong the relationship must be before the two tokens are considered a single concept

Once we have trained the phrase model we can apply it to the reviews in our corpus.  It will consider the multiworded tokens to be single phrases.

The gensim library will help us with phrase modeling, specifically the Phrases class.

In [4]:
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.models.word2vec import LineSentence

In [3]:
%mv beerAdvocateScraper/beerAdvocateScraper/BeerAdvocateReviews.csv ./intermediate/

In [5]:
import re
pattern = re.compile('^[\d,"]+')

def punct_space(token):
    """
    helper function to eliminate tokens
    that are pure punctuation or whitespace
    """
    
    return token.is_punct or token.is_space

def line_review(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    count = 0
    with open(filename, encoding='utf_8') as f:
        for review in f:
            try:
                yield re.split(pattern, review)[1].replace('\\n', '\n')
            except: pass
            if count % 1000 == 0:
                print(f'on review {count}')
            count += 1
            
def lemmatized_sentence_corpus(filename):
    """
    generator function to use spaCy to parse reviews,
    lemmatize the text, and yield sentences
    """
    
    for parsed_review in nlp.pipe(line_review(filename),
                                  batch_size=1000, n_threads=4):
        
        for sent in parsed_review.sents:
            yield u' '.join([token.lemma_ for token in sent
                             if not punct_space(token)])

In [6]:
intermediate_directory = os.path.join('.', 'intermediate')
# review_txt_filepath = os.path.join(intermediate_directory,'BeerAdvocateReviews.csv')
review_txt_filepath = os.path.join(intermediate_directory,'slimReviews.csv')
ngram_all = os.path.join(intermediate_directory, 'ngram_all')
unigram_sentences_filepath = os.path.join(ngram_all,
                                          'unigram_sentences_all.txt')

We will use `lemmatized_sentence_corpus` generator to loop over the original review text, segmenting the reviews into individual sentences and normalizing the text.  We will write this data back out to a new file (`unigram_sentence_all`), with one normalized sentence per line.  We will use this data for learning our phrase models.

In [7]:
with open(unigram_sentences_filepath, 'w', encoding='utf_8') as f:
    for sentence in lemmatized_sentence_corpus(review_txt_filepath):
        print(sentence)
        f.write(sentence + '\n')

on review 0
black and thick
tan head
nose be perfect with huge maple syrup and coffee
bourbon and vanilla
good breakfast ever
taste maple vanilla bourbon and oak
everything -PRON- want in a beer
not overly roasted
just perfect
full bodied
good carbonation
end with maple syrup
amazing
that be all
can not believe -PRON- live up to the hype
oz
bottle pour into a snifter
no bottle number
appearance inky black
stain the glass again
big and dark head than mornin delight and assassin
great surface wisps
smell bourbon
maple
vanilla
dark chocolate
everything -PRON- would want in a stout
the bourbon do take over though
taste bourbon be sharp
coffee come out here
dark chocolate
maple come out in the back
sweetness linger
bourbon flavor linger also
mouthfeel very thick
boozy
sweet with medium carbonation
overall tough to argue which be good but bourbon always win
serve alongside mornin delight and assassin
the beer be near black with a filmy beige head
all hype and bias aside this aroma be silly g

Our data in the `unigram_sentences_all` file is now organized as a large text file with one sentence per line.  This format allows us to use gensim's LineSentence class, a convenient iterator for working with gensim's other components.  It *streams* the documents/sentences from disk, so you never have to hold the entire corpus in RAM at once.  This allows you to scale the modeling to a very large corpora.

In [8]:
unigram_sentences = LineSentence(unigram_sentences_filepath)

In [9]:
for unigram_sentence in it.islice(unigram_sentences, 5,7):
    print(u' '.join(unigram_sentence))
    print(u' ')

taste maple vanilla bourbon and oak
 
everything -PRON- want in a beer
 


In [10]:
bigram_model_file = os.path.join(ngram_all, 'bigram_model')

In [11]:
if 1==1:
    phrases = Phrases(unigram_sentences)
    bigram_model = Phraser(phrases)
    bigram_model.save(bigram_model_file)
    
bigram_model = Phrases.load(bigram_model_file)

In [12]:
bigram_sentences_filepath = os.path.join(ngram_all,'bigram_sentences_all.txt')

In [13]:
with open(bigram_sentences_filepath, 'w', encoding='utf_8') as f:

    for unigram_sentence in unigram_sentences:

        bigram_sentence = u' '.join(bigram_model[unigram_sentence])

        f.write(bigram_sentence + '\n')

In [14]:
bigram_sentences = LineSentence(bigram_sentences_filepath)

In [15]:
for bigram_sentence in it.islice(bigram_sentences, 5, 7):
    print(u' '.join(bigram_sentence))
    print(u'')

taste maple vanilla bourbon and oak

everything -PRON- want in a beer



In [16]:
trigram_model_filepath = os.path.join(ngram_all, 'trigram_model')

In [17]:
if 1 == 1:

    phrases_trigram = Phrases(bigram_sentences)
    trigram_model = Phraser(phrases_trigram)
    trigram_model.save(trigram_model_filepath)
    
# load the finished model from disk
trigram_model = Phrases.load(trigram_model_filepath)

In [18]:
if 1==1:
    phrases = Phrases(unigram_sentences)
    bigram_model = Phraser(phrases)
    bigram_model.save(bigram_model_file)
    
bigram_model = Phrases.load(bigram_model_file)

In [19]:
trigram_sentences_filepath = os.path.join(ngram_all, 'trigram_sentences_all.txt')

In [20]:
with open(trigram_sentences_filepath, 'w', encoding='utf_8') as f:

    for bigram_sentence in bigram_sentences:

        trigram_sentence = u' '.join(trigram_model[bigram_sentence])

        f.write(trigram_sentence + '\n')

In [21]:
trigram_sentences = LineSentence(trigram_sentences_filepath)

In [22]:
for trigram_sentence in it.islice(trigram_sentences, 3, 5):
    print(u' '.join(trigram_sentence))
    print(u'')

bourbon and vanilla

good breakfast ever



In [23]:
trigram_reviews_filepath = os.path.join(intermediate_directory,
                                        'trigram_transformed_reviews_all.txt')

In [24]:
from spacy.lang.en.stop_words import STOP_WORDS

In [25]:

with open(trigram_reviews_filepath, 'w', encoding='utf_8') as f:

    for parsed_review in nlp.pipe(line_review(review_txt_filepath),
                                  batch_size=10000, n_threads=4):

        # lemmatize the text, removing punctuation and whitespace
        unigram_review = [token.lemma_ for token in parsed_review
                          if not punct_space(token)]

        # apply the first-order and second-order phrase models
        bigram_review = bigram_model[unigram_review]
        trigram_review = trigram_model[bigram_review]

        # remove any remaining stopwords
        trigram_review = [term for term in trigram_review
                          if term not in STOP_WORDS]
        
        # remove pronouns
        trigram_review = [term for term in trigram_review
                          if term !='-PRON-']
        

        # write the transformed review as a line in the new file
        trigram_review = u' '.join(trigram_review)
        f.write(trigram_review + '\n')

on review 0


In [91]:
print(u'Original:' + u'\n')

for review in it.islice(line_review(review_txt_filepath), 2, 3):
    print(review)

print(u'----' + u'\n')
print(u'Transformed:' + u'\n')

with open(trigram_reviews_filepath, encoding='utf_8') as f:
    for review in it.islice(f, 2, 3):
#         for id in it.islice(get_beerID(review_txt_filepath),2,3):
            print(id)
            print(review)

Original:

on review 0
Served alongside Mornin' Delight and Assassin. The beer is near black with a filmy beige head. All hype and bias aside, this aroma is silly good. Oak, roast, coffee, vanilla. World class from the first sip. The definition of grace and balance. Silky smooth on the palate. Absolutely amazing."

----

Transformed:

78820
serve alongside mornin delight assassin beer near black filmy beige head hype bias aside aroma silly good oak roast coffee vanilla world class sip definition grace balance silky smooth palate absolutely amazing



Exception ignored in: <generator object line_review at 0x11c281db0>
RuntimeError: generator ignored GeneratorExit
Exception ignored in: <generator object get_beerID at 0x11c281db0>
RuntimeError: generator ignored GeneratorExit


In [90]:
def get_beerID(filename):
    """
    generator function to read in reviews from the file
    and un-escape the original line breaks in the text
    """
    with open(review_txt_filepath, encoding='utf_8') as f:
        for review in f:
            try:
                yield re.split('(^[\d"]+)', review)[1].replace('\\n', '\n')
            except: pass

## Word2Vec

The goal of *word vector embedding models*, or *word vector models* for short is to learn dense numerical representations of each term in a corpus vocabulary.  If the model is succesful, the vectors it learns should encode some information about the *meaning* or *concept* the term represents, and the relationship between it and other terms in the vocabulary.  Word vector models are fully unsupervised &mdash; they learn all of these meaning and relationships solely by analyzing the text of the corpus, without any advanced knowledge provided.

In [28]:
from gensim.models import Word2Vec

# run with or without stop words removed and word lemmatized
trigram_sentences = LineSentence(trigram_reviews_filepath)
# trigram_sentences = LineSentence(trigram_sentences_filepath)
word2vec_filepath = os.path.join(intermediate_directory, 'word2vec_model_all')

In [29]:
if 1==1:
    
    # initiate the model and perform one epoch of training
    beer2vec = Word2Vec(trigram_sentences, size=100, window=5,
                       min_count=1, sg=1) #workers=?
    beer2vec.save(word2vec_filepath)
    
    #perform the next n epochs of training
    for i in range(1, 5):
        beer2vec.train(trigram_sentences, total_examples=beer2vec.corpus_count, epochs=2)
        beer2vec.save(word2vec_filepath)
        
        
#load the finished model from disk
beer2vec = Word2Vec.load(word2vec_filepath)
beer2vec.init_sims()

print(f'Trained model for {beer2vec.train_count} epochs.')

Trained model for 5 epochs.


In [30]:

# build a list of the terms, integer indices,
# and term counts from the food2vec model vocabulary
ordered_vocab = [(term, voc.index, voc.count)
                 for term, voc in beer2vec.wv.vocab.items()]

# sort by the term counts, so the most common terms appear first
ordered_vocab = sorted(ordered_vocab, key=lambda tup: -1*tup[-1])

# # unzip the terms, integer indices, and counts into separate lists
ordered_terms, term_indices, term_counts = zip(*ordered_vocab)

# # create a DataFrame with the food2vec vectors as data,
# # and the terms as row labels
word_vectors = pd.DataFrame(beer2vec.wv.syn0norm[term_indices, :],
                            index=ordered_terms)

word_vectors.head(10)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
coffee,0.012028,0.097096,-0.077724,0.213747,0.023751,0.096958,-0.033022,-0.101030,0.237131,-0.145274,...,-0.175268,-0.139448,-0.022495,-0.154654,-0.006476,-0.022580,-0.099448,-0.069665,-0.008444,0.216365
bourbon,0.006706,0.094473,-0.073777,0.211184,0.021388,0.088473,-0.024167,-0.102111,0.239401,-0.150555,...,-0.170733,-0.134100,-0.015045,-0.158621,-0.011447,-0.011696,-0.092141,-0.064788,-0.016234,0.224218
chocolate,0.015707,0.091595,-0.082576,0.209546,0.029002,0.098617,-0.025143,-0.097095,0.235295,-0.144275,...,-0.169356,-0.138224,-0.015839,-0.152854,-0.003384,-0.015999,-0.093416,-0.074486,-0.005449,0.220004
beer,0.008668,0.093705,-0.074859,0.215875,0.021551,0.093600,-0.033044,-0.096184,0.244431,-0.138483,...,-0.167888,-0.134354,-0.016564,-0.157530,-0.009302,-0.022340,-0.100736,-0.067297,-0.012181,0.220440
black,0.017581,0.095586,-0.078170,0.215958,0.026647,0.085226,-0.033580,-0.088818,0.240017,-0.138578,...,-0.169262,-0.143153,-0.020515,-0.156416,-0.000201,-0.022687,-0.102088,-0.073409,-0.010287,0.222813
maple_syrup,0.023244,0.101782,-0.083169,0.215741,0.032382,0.098567,-0.022170,-0.104239,0.245453,-0.135163,...,-0.166706,-0.146910,-0.018001,-0.154330,-0.004767,-0.024507,-0.105899,-0.076941,-0.014044,0.223670
maple,0.018604,0.090412,-0.081401,0.215012,0.023898,0.087188,-0.021745,-0.094199,0.244242,-0.148566,...,-0.176696,-0.147416,-0.024721,-0.155693,-0.012190,-0.015902,-0.093868,-0.069291,-0.005753,0.227276
dark,0.007636,0.088682,-0.069289,0.220537,0.033120,0.093210,-0.022818,-0.090563,0.230376,-0.132074,...,-0.178352,-0.145875,-0.016402,-0.159323,-0.014324,-0.015554,-0.104589,-0.080708,-0.008988,0.229114
thick,0.005687,0.101507,-0.065484,0.216440,0.029358,0.080772,-0.027001,-0.097177,0.243524,-0.133762,...,-0.162760,-0.130723,-0.010028,-0.151420,-0.002267,-0.023116,-0.103739,-0.060929,-0.011244,0.217321
head,0.012661,0.090200,-0.066684,0.222662,0.028686,0.097836,-0.034139,-0.097054,0.241401,-0.138868,...,-0.173682,-0.138941,-0.011796,-0.153245,-0.006603,-0.019848,-0.092079,-0.068473,-0.009687,0.227955


In [49]:
word_vectors.shape

(239, 100)

# Dimension Reduction Using t-SNE 

t-Distributed Stochastic Neighbor Embedding, or *t-SNE*, is a dimensionality reduction technique to assist with visualizing high-dimensional datasets.  It attempts to map high-dimensional data onto a low (2 or 3) dimensional  representation such that the relative distance between points are preserved as closely as possible in both high and low dimensional space.

In [31]:
from sklearn.manifold import TSNE

In [39]:
from spacy.lang.en.stop_words import STOP_WORDS

tsneInput = word_vectors.drop(STOP_WORDS, errors=u'ignore')
tsneInput = tsneInput.head(20)

In [40]:
tsneInput.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
coffee,0.012028,0.097096,-0.077724,0.213747,0.023751,0.096958,-0.033022,-0.101030,0.237131,-0.145274,...,-0.175268,-0.139448,-0.022495,-0.154654,-0.006476,-0.022580,-0.099448,-0.069665,-0.008444,0.216365
bourbon,0.006706,0.094473,-0.073777,0.211184,0.021388,0.088473,-0.024167,-0.102111,0.239401,-0.150555,...,-0.170733,-0.134100,-0.015045,-0.158621,-0.011447,-0.011696,-0.092141,-0.064788,-0.016234,0.224218
chocolate,0.015707,0.091595,-0.082576,0.209546,0.029002,0.098617,-0.025143,-0.097095,0.235295,-0.144275,...,-0.169356,-0.138224,-0.015839,-0.152854,-0.003384,-0.015999,-0.093416,-0.074486,-0.005449,0.220004
beer,0.008668,0.093705,-0.074859,0.215875,0.021551,0.093600,-0.033044,-0.096184,0.244431,-0.138483,...,-0.167888,-0.134354,-0.016564,-0.157530,-0.009302,-0.022340,-0.100736,-0.067297,-0.012181,0.220440
black,0.017581,0.095586,-0.078170,0.215958,0.026647,0.085226,-0.033580,-0.088818,0.240017,-0.138578,...,-0.169262,-0.143153,-0.020515,-0.156416,-0.000201,-0.022687,-0.102088,-0.073409,-0.010287,0.222813


In [41]:
tsne_filepath = os.path.join(intermediate_directory,
                             u'tsne_model')

tsne_vectors_filepath = os.path.join(intermediate_directory,
                                     u'tsne_vectors.npy')

In [45]:
import pickle

In [48]:
tsneInput.values.shape

(20, 100)

In [52]:
if 1 == 1:
    
    tsne = TSNE()
    tsne_vectors = tsne.fit_transform(tsneInput.values)
    
    with open(tsne_filepath, 'wb') as f:
        pickle.dump(tsne, f)

    pd.np.save(tsne_vectors_filepath, tsne_vectors)
    
with open(tsne_filepath, 'rb') as f:
    tsne = pickle.load(f)
    
tsne_vectors = pd.np.load(tsne_vectors_filepath)

tsne_vectors = pd.DataFrame(tsne_vectors,
                            index=pd.Index(tsneInput.index),
                            columns=[u'x_coord', u'y_coord'])

In [53]:
tsne_vectors[u'word'] = tsne_vectors.index

In [54]:
tsne_vectors.head()

,x_coord,y_coord,word
coffee,-65.032639,28.823454,coffee
bourbon,59.224350,21.084337,bourbon
chocolate,-16.854601,-95.139938,chocolate
beer,-57.379971,104.477707,beer
black,83.488785,92.597794,black
